<a href="https://colab.research.google.com/github/KyokoM/tsunodalab/blob/master/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#GPUの確認
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http

In [0]:
#googledriveに認証接続
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
#colabにfile取り込み
upload_filename = 'value_data.csv'#train_data.csv

In [0]:
file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

In [0]:
file_id = None
for file in file_list:
  if file.get('name') == upload_filename:
    file_id = file.get('id')
    break

if file_id is None:
  # ファイル ID を取得できなかった場合はエラーメッセージを出力します。
  print(upload_filename + ' が見つかりません.')
else:
  # colab 環境へファイルをアップロードします。
  with open(upload_filename, 'wb') as f:
    request = drive_service.files().get_media(fileId=file_id)
    media = googleapiclient.http.MediaIoBaseDownload(f, request)

    done = False
    while not done:
      progress_status, done = media.next_chunk()
      print(100*progress_status.progress(), end="")
      print("%完了")

  print('GoogleドライブからColab環境へのファイル取り込みが完了しました.')

100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


In [0]:
ls

adc.json  sample_data/  train_data.csv  value_data.csv


In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import pandas as pd
import h5py
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.svm import LinearSVC

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image, display_png
#import pydotplus as pdp

from sklearn.ensemble import RandomForestClassifier

import seaborn as sns

In [0]:
ls

adc.json                                                sample_data/
answer.csv                                              train_data.csv
GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct  train_raw_data.csv


In [0]:
#dic_type={}
#dic_type[1]="str"
#dic_type[2]="str"
#for i in range(3,11691):
 #   dic_type[2]="float64"
#ft2=pd.read_table("GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct", sep="\t", header=2,dtype=dic_type)

In [0]:
#ft2.to_csv("train_raw_data.csv")
#del ft2
#pd.read_csv("train_raw_data.csv").transpose().to_csv("train_raw_data_transpose.csv")

In [0]:
#ft2=ft2.transpose()

In [0]:
#ft1_new = ft2.drop("Description", axis=0).drop("Name", axis=0)

#df2=pd.read_csv("answer.csv", delimiter="\t")

In [0]:
#merge=pd.merge(df2, ft1,left_on="SAMPID",right_index=True)

In [0]:
#df3=merge.iloc[:,:2]

In [0]:
#ft3=merge[:].drop("SMTS", axis=1)

In [0]:
#label = LabelEncoder()
#ラベルを覚えさせる
#label = label.fit(df3['SMTS'])
#ラベルを整数に変換
#df3['tissue'] = label.transform(df3['SMTS'])
#df3.head()

In [0]:
ls

sample_data/


In [0]:
ft5 = np.loadtxt("train_data.csv",delimiter=",", skiprows=1)[1:]

NameError: ignored

In [0]:
#ft5=pd.read_csv("train_data.csv", delimiter=",",dtype="float16",index_col=0)

KeyboardInterrupt: ignored

In [0]:
print(ft5.head())

            0          1         2  ...    56199     56200     56201
3.0  0.108215  21.406250  0.160156  ...   6856.0  0.932129  0.000000
4.0  0.081787  15.031250  0.000000  ...  26016.0  6.339844  6.156250
5.0  0.000000  17.531250  0.000000  ...   8376.0  1.025391  0.994629
6.0  0.000000  23.812500  0.000000  ...   8680.0  0.000000  0.000000
7.0  0.022705   7.441406  0.151367  ...  29088.0  0.000000  1.139648

[5 rows x 56202 columns]


In [0]:
df5 = np.loadtxt("value_data.csv",delimiter=",", skiprows=1)[1:]

In [0]:
#df4=pd.read_csv("value_data.csv", delimiter=",",dtype="float16",index_col=0).values

In [0]:
X_train, X_test, y_train, y_test =train_test_split(ft5.values,df4.values)
print(ft5.shape,X_train.shape,y_train.shape)
print(df4.shape,X_test.shape,y_test.shape)

(11688, 56202) (8766, 56202) (8766, 1)
(11688, 1) (2922, 56202) (2922, 1)


In [0]:
print((X_test))

TypeError: ignored

In [0]:
clf_lr = LogisticRegression(solver='lbfgs', C=1,  multi_class= "auto", max_iter=1000)#モデルの生成
clf_lr.fit(X_train, y_train)#モデルの生成
lr_p= clf_lr.predict(X_test)#テストデータに伴う予測値
ac_score_lr = metrics.accuracy_score(y_test, lr_p)
print(ac_score_lr)

NameError: ignored

In [0]:
print(ft5.values)

[[1.082e-01 2.141e+01 1.602e-01 ... 6.856e+03 9.321e-01 0.000e+00]
 [8.179e-02 1.503e+01 0.000e+00 ... 2.602e+04 6.340e+00 6.156e+00]
 [0.000e+00 1.753e+01 0.000e+00 ... 8.376e+03 1.025e+00 9.946e-01]
 ...
 [7.794e-02 1.035e+01 0.000e+00 ... 1.885e+04 0.000e+00 2.607e+00]
 [1.181e-01 6.262e+00 1.312e-01 ... 3.622e+04 4.731e+01 4.445e+00]
 [0.000e+00 7.699e+00 0.000e+00 ... 2.752e+04 3.277e+00 6.362e-01]]


In [0]:
df5=np.identity(30)[df4.values.flatten().astype(int)]

In [0]:
X_train, X_test, y_train, y_test =train_test_split(ft5,df5)
print(ft4.shape,X_train.shape,y_train.shape)
print(df4.shape,X_test.shape,y_test.shape)

(11688, 56203) (8766, 56202) (8766, 30)
(11688, 1) (2922, 56202) (2922, 30)


In [0]:
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import math
n_in = 56202
n_out = 30
x = tf.placeholder(tf.float32,[None,n_in])
t = tf.placeholder(tf.float32,[None,n_out])
is_training = tf.placeholder(tf.bool)

#モデル構築
#W = tf.Variable(tf.zeros([n_in, n_out]))
#b = tf.Variable(tf.zeros([n_out]))
#y = tf.matmul(x, W) + b


#####
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        self.W = tf.Variable(tf.random_uniform(shape=(in_dim, out_dim), minval=-0.08, maxval=0.08), name='W')
        self.b = tf.Variable(tf.zeros(out_dim), name='b')
        self.function = function
        
        self.params = [self.W, self.b]
    
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

def sgd(cost, params, eta=0.1):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        updates.append(param.assign_sub(eta * grad))
    return updates

def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
dropout_keep_prob = 0.5
class Dropout:
    def __init__(self, dropout_keep_prob=1.0):
        self.dropout_keep_prob = dropout_keep_prob
        self.params = []
    
    def __call__(self, x):
        # 訓練時のみdropoutを適用
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, keep_prob=self.dropout_keep_prob),
            false_fn=lambda: x
        )

layers = [
    Dense(56202, 1000, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(1000, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 30, tf.nn.softmax)
]
params = []
h = x
for layer in layers:
    h = layer(h)
    params += layer.params
y = h

cost = - tf.reduce_mean(tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y), axis=1))


updates = sgd(cost, params)
train = tf.group(*updates)
###
#cross_entropy = tf.reduce_mean(
#    tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y))
#train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)




n_epochs = 10
batch_size = 1000
n_batches = math.ceil(len(X_train) / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(n_epochs):
    X_train, y_train = shuffle(X_train, y_train)
    print(X_train.shape)
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        #print(X_train.shape,start,end)
        sess.run(train, feed_dict={x: X_train[start:end], t: y_train[start:end], is_training: True})
        #print(X_train.shape)
        y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: X_test, t: y_test, is_training: False})
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost_valid_,
            accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))
        ))
    #loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: X_test, t: y_test})
    #print('epoch: {:2}, loss: {:.5f}, accuracy: {:.5f}'.format(epoch, loss, acc))
        #y_pred, cost_valid_ = sess.run([cross_entropy, cost], feed_dict={x: X_test, t: y_test, is_training: False})
        #print('epoch: {:2}, loss: {:.5f}, accuracy: {:.5f}'.format(epoch,cost_valid_,y_pred, cost_valid_))
   # print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
    #      epoch + 1,
     #     cost_valid_,
     #     accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    #))



(8766, 56202)
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 1, Valid Cost: nan, Valid Accuracy: 0.073
(8766, 56202)
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 2, Valid Cost: nan, Valid Accuracy: 0.073
(8766, 56202)
EPOCH: 3, Valid Cost: nan, Valid Accuracy: 0.073
EPOCH: 3, Valid Cost: nan, 

In [0]:
print(X_train.shape,y_train.shape)

(8766, 56202) (8766, 30)


In [0]:
print(X_train)

[[7.513e-02 3.528e+01 8.344e-02 ... 6.780e+03 0.000e+00 9.424e-01]
 [0.000e+00 2.023e+01 1.407e-01 ... 2.045e+04 1.637e+00 1.589e+00]
 [2.124e-02 1.566e+01 9.436e-02 ... 2.674e+04 2.195e+00 2.133e+00]
 ...
 [2.170e-01 1.315e+01 1.071e-01 ... 3.098e+04 2.492e+00 3.629e+00]
 [1.279e-01 7.828e+00 1.421e-01 ... 3.690e+04 1.653e+00 1.605e+00]
 [0.000e+00 1.525e+01 2.144e-01 ... 2.019e+04 5.816e+00 5.648e+00]]
